# Graph Neural Networks with Pytorch

Original Paper: https://arxiv.org/abs/1706.02216    
Original Code: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/reddit.py

In [1]:
!pip install pytorch torchvision torchaudio
!pip install torch_scatter
!pip install torch_sparse

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torchvision-0.14.1-cp310-cp310-win_amd64.whl (1.1 MB)
  Using cached torchaudio-0.13.1-cp310-cp310-win_amd64.whl (2.0 MB)
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch: started
  Running setup.py install for pytorch: finished with status 'error'

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\wikid\AppData\Local\Temp\pip-install-ix6_d1lf\pytorch_b8dff50a61f54f5994ecda1524a19b9c\setup.py", line 15, in <module>
          raise Exception(message)
      Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
  error: subprocess-exited-with-error
  
  × Running setup.py install for pytorch did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in 


[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.1.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import sys

import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.datasets import Reddit
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv

sys.path.append('./')
from utils import *
logger = make_logger(name='graphsage_logger')

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
from graphSAGE import SAGE

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)


In [ ]:
path = os.path.join(os.getcwd(), '..', 'data', 'Reddit')
dataset = Reddit(path)
data = dataset[0]

Extracting c:\Users\wikid\Desktop\data\Reddit\raw\reddit.zip
Processing...
Done!


In [ ]:
logger.info(f"Node Feature Matrix Info: # Nodes: {data.x.shape[0]}, # Node Features: {data.x.shape[1]}")

logger.info(f"Edge Index Shape: {data.edge_index.shape}")
logger.info(f"Edge Weight: {data.edge_attr}")

print(data.train_mask.sum().item())

2023-03-07 10:28:16,964 - graphsage_logger - Node Feature Matrix Info: # Nodes: 232965, # Node Features: 602
2023-03-07 10:28:16,976 - graphsage_logger - Edge Index Shape: torch.Size([2, 114615892])
2023-03-07 10:28:16,978 - graphsage_logger - Edge Weight: None


153431


In [ ]:
# Define Sampler
train_loader = NeighborSampler(
    data.edge_index, node_idx=data.train_mask,
    sizes=[25, 10], batch_size=1024, shuffle=True, num_workers=12)

subgraph_loader = NeighborSampler(
    data.edge_index, node_idx=None,
    sizes=[-1], batch_size=1024, shuffle=False, num_workers=12)

c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.NeighborSampler' is deprecated, use 'loader.NeighborSampler' instead
  warnings.warn(out)
c:\Users\wikid\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:554: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
batch_size, n_id, adjs = next(iter(train_loader))

logger.info(f"Current Batch Size: {batch_size}")
logger.info(f"현재 Subgraph에서 사용된 모든 node id의 개수: {n_id.shape[0]}")
logger.info(f"Layer의 수: {len(adjs)}")

A = adjs[1].size[0] - batch_size
B = adjs[0].size[0] - A - batch_size

logger.info(f"진행 방향: {B}개의 2-hop neighbors ->"
            f"{A}개의 1-hop neighbors -> {batch_size}개의 Head Nodes")

2023-03-07 10:29:46,596 - graphsage_logger - Current Batch Size: 1024
2023-03-07 10:29:46,600 - graphsage_logger - 현재 Subgraph에서 사용된 모든 node id의 개수: 106532
2023-03-07 10:29:46,601 - graphsage_logger - Layer의 수: 2
2023-03-07 10:29:46,603 - graphsage_logger - 진행 방향: 84747개의 2-hop neighbors ->20761개의 1-hop neighbors -> 1024개의 Head Nodes


In [ ]:
# # 모델 정의
# class SAGE(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(SAGE, self).__init__()

#         self.num_layers = 2

#         self.convs = torch.nn.ModuleList()
#         self.convs.append(SAGEConv(in_channels, hidden_channels))
#         self.convs.append(SAGEConv(hidden_channels, out_channels))

#     def forward(self, x, adjs):
#         for i, (edge_index, _, size) in enumerate(adjs):
#             x_target = x[:size[1]]
#             x = self.convs[i]((x, x_target), edge_index)

#             if i != self.num_layers - 1:
#                 x = F.relu(x)
#                 x = F.dropout(x, p=0.5, training=self.training)
#         return x.log_softmax(dim=-1)

#     def inference(self, x_all):
#         pbar = tqdm(total=x_all.size(0) * self.num_layers)
#         pbar.set_description('Evaluating')

#         for i in range(self.num_layers):
#             xs = []
#             for batch_size, n_id, adj in subgraph_loader:
#                 edge_index, _, size = adj.to(device)
#                 x = x_all[n_id].to(device)
#                 x_target = x[:size[1]]
#                 x = self.convs[i]((x, x_target), edge_index)
#                 if i != self.num_layers - 1:
#                     x = F.relu(x)
#                 xs.append(x.cpu())

#                 pbar.update(batch_size)

#             x_all = torch.cat(xs, dim=0)

#         pbar.close()
#         return x_all

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGE(dataset.num_features, 256, dataset.num_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

x = data.x.to(device)
y = data.y.squeeze().to(device)

NameError: name 'SAGE' is not defined

In [15]:
# 학습
class GraphSAGE(model, optimizer, train_loader, device, data):

    def __init__(self, model, optimizer, train_loader, device, data):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.device = device
        self.data = data

    def train(self, epoch):
        self.model.train()

        pbar = tqdm(total=int(self.data.train_mask.sum()))
        pbar.set_description(f'Epoch {epoch:02d}')

        total_loss = total_correct = 0
        for batch_size, n_id, adjs in self.train_loader:
            adjs = [adj.to(self.device) for adj in adjs]

            self.optimizer.zero_grad()
            out = self.model(x[n_id], adjs)
            loss = F.nll_loss(out, y[n_id[:batch_size]])
            loss.backward()
            self.optimizer.step()

            total_loss += float(loss)
            total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
            pbar.update(batch_size)

        pbar.close()

        loss = total_loss / len(self.train_loader)
        approx_acc = total_correct / int(self.data.train_mask.sum())
        return loss, approx_acc


    # 모델 평가
    @torch.no_grad()
    def test(self):
        self.model.eval()
        out = self.model(x)

        y_true = y.cpu().unsqueeze(-1)
        y_pred = out.argmax(dim=-1, keepdim=True)

        results = []
        for mask in [self.data.train_mask, self.data.val_mask, self.data.test_mask]:
            results += [int(y_pred[mask].eq(y_true[mask]).sum()) / int(mask.sum())]
        return results

In [ ]:
for epoch in range(1, 3):
    loss, acc = graph_sage.train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')
    train_acc, val_acc, test_acc = graph_sage.test()
    print(f'Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')


AttributeError: type object 'GraphSAGE' has no attribute 'model'